In [6]:
 from __future__ import print_function, division

#from keras.datasets import mnist
from keras.layers import Input, Dense, Reshape, Flatten, Dropout, AveragePooling2D, GaussianNoise
from keras.layers import BatchNormalization, Activation, ZeroPadding2D,Conv2DTranspose
from keras.layers.advanced_activations import LeakyReLU
from keras.layers.convolutional import UpSampling2D, Conv2D
from keras.models import Sequential, Model
from keras.optimizers import Adam
import matplotlib.pyplot as plt
from tqdm import notebook
import os
import sys
import numpy as np


In [7]:
def load_data():
  data_x_p1 = np.load("/content/drive/MyDrive/UHCourse/ECE6397/mldata_256/Data_x_Bacteria.npy")
  #data_x_p2 = np.load("/content/drive/MyDrive/UHCourse/ECE6397/mldata/Data_x_Normal.npy")
  #data_x_p3 = np.load("/content/drive/MyDrive/UHCourse/ECE6397/mldata/Data_x_Virus.npy")
  #data_x_p4 = np.load("/content/drive/MyDrive/UHCourse/ECE6397/mldata/Data_x_Covid.npy")
  #X_Train = np.concatenate((data_x_p1,data_x_p2,data_x_p3,data_x_p4),axis = 0)

  data_y_p1 = np.load("/content/drive/MyDrive/UHCourse/ECE6397/mldata_256/Data_y_Bacteria.npy")
  #data_y_p2 = np.load("/content/drive/MyDrive/UHCourse/ECE6397/mldata/Data_y_Normal.npy")
  #data_y_p3 = np.load("/content/drive/MyDrive/UHCourse/ECE6397/mldata/Data_y_Virus.npy")
  #data_y_p4 = np.load("/content/drive/MyDrive/UHCourse/ECE6397/mldata/Data_y_Covid.npy")
  #y_Train = np.concatenate((data_y_p1,data_y_p2,data_y_p3,data_y_p4),axis = 0)

  #return X_Train, y_Train
  #X_mean = np.mean(data_x_p1,axis=-1)
  X_0 = data_x_p1[:,:,:,0]

  return X_0, data_y_p1

In [11]:
class DCGAN():
    def __init__(self):
        # Input shape
        self.img_rows =   256 #@param  # 图像的高
        self.img_cols =   256 #@param  # 图像的宽
        self.channels =     1 #@param  # 彩色图像
        self.img_shape = (self.img_rows, self.img_cols, self.channels)
        self.latent_dim = 4096  #@param
        self.init_shape =    100#@param  # gen
        self.init_Name  = "Covid" #@param

        # 设置学习率为0.0002
        optimizer = Adam(0.0002, 0.5)

        # Build and compile the discriminator
        self.discriminator = self.build_discriminator()
        self.discriminator.compile(loss='binary_crossentropy',
            optimizer=optimizer,
            metrics=['accuracy'])

        # Build the generator
        self.generator = self.build_generator()

        # The generator takes noise as input and generates imgs
        z = Input(shape=(self.init_shape,))
        img = self.generator(z)

        # For the combined model we will only train the generator
        self.discriminator.trainable = False

        # The discriminator takes generated images as input and determines validity
        valid = self.discriminator(img)

        # The combined model  (stacked generator and discriminator)
        # Trains the generator to fool the discriminator
        self.combined = Model(z, valid)
        self.combined.compile(loss='binary_crossentropy', optimizer=optimizer)
    def build_generator(self):
        kernel_size_G = [4,4,4,3,3,3,3,3] #@param
        _momentum_G = 0.7 #@param
        model = Sequential()

        model.add(Dense(256*4*4, activation="relu", input_dim=self.init_shape))
        
        #1x1x4096 
        model.add(Reshape((4, 4, 256)))
        #model.add(Conv2D(filters = 256, kernel_size = kernel_size_G[0]))
        #model.add(Activation('relu'))


        #4x4x256 - kernel sized increased by 1
        model.add(Conv2D(256, kernel_size=kernel_size_G[1], padding="same"))
        model.add(BatchNormalization(momentum=_momentum_G))
        model.add(Activation("relu"))
        model.add(UpSampling2D())
        
        #8x8x256 - kernel sized increased by 1
        model.add(Conv2D(256, kernel_size=kernel_size_G[2], padding="same"))
        model.add(BatchNormalization(momentum=_momentum_G))
        model.add(Activation("relu"))
        model.add(UpSampling2D())

        #16x16x128
        model.add(Conv2D(128, kernel_size=kernel_size_G[3], padding="same"))
        model.add(BatchNormalization(momentum=_momentum_G))
        model.add(Activation("relu"))
        model.add(UpSampling2D())
        
        #32x32x64
        model.add(Conv2D(64, kernel_size=kernel_size_G[4], padding="same"))
        model.add(BatchNormalization(momentum=_momentum_G))
        model.add(Activation("relu"))
        model.add(UpSampling2D())
        
        #64x64x32
        model.add(Conv2D(32, kernel_size=kernel_size_G[4], padding="same"))
        model.add(BatchNormalization(momentum=_momentum_G))
        model.add(Activation("relu"))
        model.add(UpSampling2D())

        #128x128x16
        model.add(Conv2D(32, kernel_size=kernel_size_G[5], padding="same"))
        model.add(BatchNormalization(momentum=_momentum_G))
        model.add(Activation("relu"))
        model.add(UpSampling2D())


        #256x256x8
        model.add(Conv2D(self.channels, kernel_size=kernel_size_G[6], padding="same"))
        model.add(Activation("tanh"))

        model.summary()

        noise = Input(shape=(self.init_shape,))
        img = model(noise)

        return Model(noise, img)
    def build_discriminator(self):
        filter_num = [8,16,32,64,128,256] #@param
        Kernel_size = [3,3,3,3,3,3] #@param
        Stride = [1,1,1,1,1,1] #@param
        dropout = 0.25 #@param
        _momentum = 0.7 #@param
        _alpha = 0.2 #@param
        model = Sequential()

        #0 #256x256x3 Image
        model.add(Conv2D(filter_num[0], kernel_size = Kernel_size[0], strides = Stride[0], input_shape=self.img_shape, padding="same"))
        model.add(LeakyReLU(alpha=_alpha))
        model.add(Dropout(dropout))
        model.add(AveragePooling2D())
        
        #1  #128x128x8
        #model.add(Conv2D(filter_num[1], kernel_size= Kernel_size[1], strides = Stride[1], padding="same"))
        #model.add(ZeroPadding2D(padding=((0,1),(0,1))))
        #model.add(BatchNormalization(momentum=_momentum))
        #model.add(LeakyReLU(alpha=_alpha))
        #model.add(Dropout(dropout))
        #model.add(AveragePooling2D())
        
        #2 #64x64x16
        model.add(Conv2D(filter_num[2], kernel_size= Kernel_size[2], strides = Stride[2], padding="same"))
        model.add(BatchNormalization(momentum=_momentum))
        model.add(LeakyReLU(alpha=_alpha))
        model.add(Dropout(dropout))
        model.add(AveragePooling2D())
        
        #3 32x32x32
        model.add(Conv2D(filter_num[3], kernel_size=Kernel_size[3], strides= Stride[3], padding="same"))
        model.add(BatchNormalization(momentum=_momentum))
        model.add(LeakyReLU(alpha=_alpha))
        model.add(Dropout(dropout))     
        model.add(AveragePooling2D())
        
        #4 #16x16x64
        model.add(Conv2D(filter_num[4], kernel_size=Kernel_size[4], strides= Stride[4], padding="same"))
        model.add(BatchNormalization(momentum=_momentum))
        model.add(LeakyReLU(alpha=_alpha))
        model.add(Dropout(dropout))      
        model.add(AveragePooling2D())
        
        #5 #8x8x128
        model.add(Conv2D(filter_num[5], kernel_size=Kernel_size[5], strides= Stride[5], padding="same"))
        model.add(BatchNormalization(momentum=_momentum))
        model.add(LeakyReLU(alpha=_alpha))
        model.add(Dropout(dropout))
        model.add(AveragePooling2D())

        #6 #4x4x256
        model.add(Flatten())

        #7 #256
        model.add(Dense(256))
        model.add(LeakyReLU(alpha=_alpha))       
        
        
        model.add(Dense(1, activation='sigmoid'))
        model.summary()
        img = Input(shape=self.img_shape)
        validity = model(img)
        return Model(img, validity)
    def train(self, epochs, batch_size=128, save_interval=50):
        # Load the dataset
        X_train,y1 = load_data()
        X_train = np.expand_dims(X_train, axis=3)
        index = np.shape(X_train)[1]
        pic_path = '/content/drive/MyDrive/UHCourse/ECE6397/mlresult_%d/%s_BS_'%(index,self.init_Name)+str(batch_size)+'_SI'+str(save_interval)
        try:
          os.makedirs(pic_path)
        except:
          pass
        valid = np.ones((batch_size, 1))
        fake = np.zeros((batch_size, 1))
        for epoch in notebook.tqdm(range(epochs)):
            # ---------------------
            #  Train Discriminator
            # Select a random half of images
            idx = np.random.randint(0, X_train.shape[0], batch_size)
            imgs = X_train[idx]
            # Sample noise and generate a batch of new images
            noise = np.random.normal(0, 1, (batch_size, self.init_shape))
            gen_imgs = self.generator.predict(noise)
            # Train the discriminator (real classified as ones and generated as zeros)
            d_loss_real = self.discriminator.train_on_batch(imgs, valid)
            d_loss_fake = self.discriminator.train_on_batch(gen_imgs, fake)
            d_loss = 0.5 * np.add(d_loss_real, d_loss_fake)
            # ---------------------
            #  Train Generator
            # ---------------------
            # Train the generator (wants discriminator to mistake images as real)
            g_loss = self.combined.train_on_batch(noise, valid)
            # Plot the progress
            # If at save interval => save generated image samples
            if epoch % save_interval == 0:

                self.save_imgs(epoch,pic_path)
                print ("%d [D loss: %f, acc.: %.2f%%] [G loss: %f]" % (epoch, d_loss[0], 100*d_loss[1], g_loss))
    def save_imgs(self, epoch,pic_path):
        r, c = 2, 2
        noise = np.random.normal(0, 1, (r * c, self.init_shape))
        gen_imgs = self.generator.predict(noise)
        # Rescale images 0 - 1
        gen_imgs = 0.5 * gen_imgs + 0.5
        np.save(pic_path+"/img_%d.npy" % epoch, gen_imgs)
        fig, axs = plt.subplots(r, c)
        cnt = 0
        for i in range(r):
            for j in range(c):
                plt
                #axs[i,j].imshow(gen_imgs[cnt, :,:,:])
                axs[i,j].imshow(gen_imgs[cnt, :,:,0], cmap='gray')
                axs[i,j].axis('off')
                cnt += 1
        fig.savefig(pic_path+"/img_%d.png" % epoch)
        plt.close()

In [12]:
#if __name__ == '__main__':
dcgan = DCGAN()
#  dcgan.train(epochs=10000, batch_size=50, save_interval=50)

Model: "sequential_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_24 (Conv2D)           (None, 256, 256, 8)       80        
_________________________________________________________________
leaky_re_lu_12 (LeakyReLU)   (None, 256, 256, 8)       0         
_________________________________________________________________
dropout_10 (Dropout)         (None, 256, 256, 8)       0         
_________________________________________________________________
average_pooling2d_10 (Averag (None, 128, 128, 8)       0         
_________________________________________________________________
conv2d_25 (Conv2D)           (None, 128, 128, 32)      2336      
_________________________________________________________________
batch_normalization_20 (Batc (None, 128, 128, 32)      128       
_________________________________________________________________
leaky_re_lu_13 (LeakyReLU)   (None, 128, 128, 32)     

In [13]:
dcgan.train(epochs=10000, batch_size=70, save_interval=70)

0 [D loss: 4.348557, acc.: 0.71%] [G loss: 0.504655]
70 [D loss: 4.209634, acc.: 20.71%] [G loss: 10.725351]
140 [D loss: 0.025529, acc.: 99.29%] [G loss: 0.389278]
210 [D loss: 0.000523, acc.: 100.00%] [G loss: 0.017015]
280 [D loss: 0.000307, acc.: 100.00%] [G loss: 0.005877]
350 [D loss: 0.000203, acc.: 100.00%] [G loss: 0.002324]
420 [D loss: 0.000177, acc.: 100.00%] [G loss: 0.001776]
490 [D loss: 0.000048, acc.: 100.00%] [G loss: 0.001950]
560 [D loss: 0.000067, acc.: 100.00%] [G loss: 0.001082]
630 [D loss: 0.000089, acc.: 100.00%] [G loss: 0.001469]
700 [D loss: 0.000109, acc.: 100.00%] [G loss: 0.000859]
770 [D loss: 0.000020, acc.: 100.00%] [G loss: 0.000808]
840 [D loss: 0.000074, acc.: 100.00%] [G loss: 0.000753]
910 [D loss: 0.000022, acc.: 100.00%] [G loss: 0.000794]
980 [D loss: 0.000032, acc.: 100.00%] [G loss: 0.000738]
1050 [D loss: 0.000022, acc.: 100.00%] [G loss: 0.000416]
1120 [D loss: 0.000021, acc.: 100.00%] [G loss: 0.000484]
1190 [D loss: 0.000017, acc.: 100.0

In [9]:
dcgan.train(epochs=10000, batch_size=60, save_interval=70)

0 [D loss: 0.000000, acc.: 100.00%] [G loss: 0.000000]
70 [D loss: 0.000000, acc.: 100.00%] [G loss: 0.000000]
140 [D loss: 0.000000, acc.: 100.00%] [G loss: 0.000000]
210 [D loss: 0.000000, acc.: 100.00%] [G loss: 0.000000]
280 [D loss: 0.000000, acc.: 100.00%] [G loss: 0.000000]
350 [D loss: 0.000000, acc.: 100.00%] [G loss: 0.000000]
420 [D loss: 0.000000, acc.: 100.00%] [G loss: 0.000000]
490 [D loss: 0.000000, acc.: 100.00%] [G loss: 0.000000]
560 [D loss: 0.000000, acc.: 100.00%] [G loss: 0.000000]
630 [D loss: 0.000000, acc.: 100.00%] [G loss: 0.000000]
700 [D loss: 0.000000, acc.: 100.00%] [G loss: 0.000000]
770 [D loss: 0.000000, acc.: 100.00%] [G loss: 0.000000]
840 [D loss: 0.000000, acc.: 100.00%] [G loss: 0.000000]
910 [D loss: 0.000000, acc.: 100.00%] [G loss: 0.000000]
980 [D loss: 0.000000, acc.: 100.00%] [G loss: 0.000000]
1050 [D loss: 0.000000, acc.: 100.00%] [G loss: 0.000000]
1120 [D loss: 0.000000, acc.: 100.00%] [G loss: 0.000000]
1190 [D loss: 0.000000, acc.: 10